In [1]:
from keras.datasets import imdb

2024-04-15 14:47:49.591645: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-15 14:47:49.627016: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 14:47:49.937708: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 14:47:49.940028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 14:47:50.519621: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
(train_data,train_label),(test_data,test_label) = imdb.load_data(num_words=10000)

In [3]:
import numpy as np

def vectorize_sequences(sequences,dimensions=10000):
    results = np.zeros((len(sequences),dimensions))
    for i,sequence in enumerate(sequences):
        results[i,sequence]=1
        
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)


In [4]:
y_train = np.asarray(train_label).astype('float32')
y_test= np.asarray(test_label).astype('float32')

In [5]:
from keras.models import Sequential
from keras.layers import Dense

In [6]:
model = Sequential()

model.add(Dense(128,input_shape=(10000,),activation='relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(16,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

2024-04-15 14:47:53.968407: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-15 14:47:53.985758: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1280128   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1291009 (4.92 MB)
Trainable params: 1291009 (4.92 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
history = model.fit(x_train,y_train,validation_split=0.2,epochs=20,verbose=1,batch_size=512)

Epoch 1/20
40/40 [==============================] - 1s 23ms/step - loss: 0.1375 - accuracy: 0.8166 - val_loss: 0.0833 - val_accuracy: 0.8908
Epoch 2/20
40/40 [==============================] - 1s 15ms/step - loss: 0.0538 - accuracy: 0.9312 - val_loss: 0.0881 - val_accuracy: 0.8864
Epoch 3/20
40/40 [==============================] - 1s 16ms/step - loss: 0.0343 - accuracy: 0.9597 - val_loss: 0.0988 - val_accuracy: 0.8754
Epoch 4/20
40/40 [==============================] - 1s 16ms/step - loss: 0.0288 - accuracy: 0.9663 - val_loss: 0.0979 - val_accuracy: 0.8776
Epoch 5/20
40/40 [==============================] - 1s 16ms/step - loss: 0.0190 - accuracy: 0.9793 - val_loss: 0.1015 - val_accuracy: 0.8790
Epoch 6/20
40/40 [==============================] - 1s 15ms/step - loss: 0.0132 - accuracy: 0.9871 - val_loss: 0.1033 - val_accuracy: 0.8758
Epoch 7/20
40/40 [==============================] - 1s 15ms/step - loss: 0.0109 - accuracy: 0.9895 - val_loss: 0.1049 - val_accuracy: 0.8766
Epoch 8/20
40

In [10]:
mse,mae = model.evaluate(x_test,y_test)

print('MSE ',mse)
print('MAE ',mae)

782/782 [==============================] - 1s 743us/step - loss: 0.1222 - accuracy: 0.8595
MSE  0.1222001239657402
MAE  0.8595200181007385


In [11]:
y_preds = model.predict(x_test)

782/782 [==============================] - 1s 676us/step


In [12]:
y_preds

array([[1.7964408e-01],
       [9.9999183e-01],
       [6.5062815e-01],
       ...,
       [1.7486162e-02],
       [4.4853770e-04],
       [5.1575233e-03]], dtype=float32)

In [13]:
y_test

array([0., 1., 1., ..., 0., 0., 0.], dtype=float32)

In [14]:
tests=[]
for i in y_test:
    tests.append(int(i))
preds=[]
for i in y_preds:
    if i[0]>0.5:
        preds.append(1)
    else:
        preds.append(0)
from sklearn.metrics import accuracy_score,precision_score,recall_score

print(accuracy_score(tests,preds))
print(precision_score(tests,preds))
print(recall_score(tests,preds))

0.85952
0.8548641819330385
0.86608


In [15]:
word_index = imdb.get_word_index()
def return_token(tid):
    for k,v in word_index.items():
        # We decode the review; note that our indices were offset by 3
        # because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
        if v==tid-3:
            return k
    return '?'
def print_review(id_):
    sentence = ' '.join(return_token(i) for i in train_data[id_])
    return sentence
print_review(0)

1641221/1641221 [==============================] - 0s 0us/step


"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [16]:
def print_review(id_):
    sentence = ' '.join(return_token(i) for i in train_data[id_])
    return sentence

In [17]:
print_review(0)

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [18]:
train_label[0] #Positive

1

In [19]:
print_review(1)

"? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ? the hair is big lots of boobs ? men wear those cut ? shirts that show off their ? sickening that men actually wore them and the music is just ? trash that plays over and over again in almost every scene there is trashy music boobs and ? taking away bodies and the gym still doesn't close for ? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [20]:
train_label[1] # Negative

0

In [21]:
print_review(2)

"? this has to be one of the worst films of the 1990s when my friends i were watching this film being the target audience it was aimed at we just sat watched the first half an hour with our jaws touching the floor at how bad it really was the rest of the time everyone else in the theatre just started talking to each other leaving or generally crying into their popcorn that they actually paid money they had ? working to watch this feeble excuse for a film it must have looked like a great idea on paper but on film it looks like no one in the film has a clue what is going on crap acting crap costumes i can't get across how ? this is to watch save yourself an hour a bit of your life"

In [22]:
train_label[2] # Negative

0